In [1]:
import pandas as pd
train=pd.read_csv('../input/song-popularity-prediction/train.csv')

In [2]:
test=pd.read_csv('../input/song-popularity-prediction/test.csv')

In [3]:
features=train.columns
features

In [4]:
train.head()

In [5]:
test.head()

In [6]:
train.info()

In [7]:
test.info()

# # EDA and Null Checks

In [8]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme()
train.hist(figsize=(12,10))
plt.show()

In [83]:
# song_duration_ms plot
#ax=sns.displot(data=train,x='song_duration_ms',kind='kde',color='red',height=4,aspect=3) 
ax=sns.displot(data=train,x='song_duration_ms',kind='hist',color='red',bins=50,height=4,aspect=3).set(title='TRAIN Feature: Song Duration')
ax=sns.displot(data=test,x='song_duration_ms',kind='hist',color='blue',bins=50,height=4,aspect=3).set(title='TEST Feature: Song Duration')
ax=sns.displot(data=train,x='song_duration_ms',kind='kde',height=4,aspect=3,hue='song_popularity').set(title='Target Distribution')

In [84]:
# acousticness plot
import numpy as np
ax=sns.displot(data=train,x='acousticness',kind='hist',color='orange',height=5,aspect=3).set(title='TRAIN Feature: Acousticness')
ax=sns.displot(data=test,x='acousticness',kind='hist',color='green',height=5,aspect=3).set(title='TEST Feature: Acousticness')
ax=sns.displot(data=train,x='acousticness',kind='kde',height=5,aspect=3,hue='song_popularity').set(title='Target Distribution') 

In [85]:
# danceability plot
ax=sns.displot(data=train,x='danceability',kind='hist',color='green',height=4,aspect=3).set(title='TRAIN Feature: Danceability')
ax=sns.displot(data=test,x='danceability',kind='hist',color='red',height=4,aspect=3).set(title='TEST Feature: Danceability')
ax=sns.displot(data=train,x='danceability',kind='kde',height=4,aspect=3,hue='song_popularity').set(title='Target Distribution')

In [86]:
# energy plot - FYI - #kind=kde for distribution plot
ax=sns.displot(data=train,x='energy',kind='hist',color='purple',height=4,aspect=3).set(title='TRAIN Feature: Energy')         
ax=sns.displot(data=test,x='energy',kind='hist',color='orange',height=4,aspect=3).set(title='TEST Feature: Energy')
ax=sns.displot(data=train,x='energy',kind='kde',height=4,aspect=3,hue='song_popularity').set(title='Target Distribution')

In [87]:
# instrumentalness plot
ax=sns.displot(data=train,x='instrumentalness',kind='kde',color='brown',height=4,aspect=3).set(title='TRAIN Feature: Instrumentalness') 
ax=sns.displot(data=train,x=np.log(train['instrumentalness']),kind='hist',color='brown',bins=30,height=4,aspect=3).set(title='TRAIN Feature: Log Instrumentalness')
ax=sns.displot(data=test,x='instrumentalness',kind='kde',color='darkblue',height=4,aspect=3).set(title='TEST Feature: Instrumentalness')
ax=sns.displot(data=train,x=np.log(test['instrumentalness']),kind='hist',color='darkblue',bins=30,height=4,aspect=3).set(title='TEST Feature: Log Instrumentalness')
ax=sns.displot(data=train,x='instrumentalness',kind='kde',height=4,aspect=3,hue='song_popularity').set(title='Target Distribution')

In [88]:
# key plot
sns.displot(data=train,x='key',color='darkblue',height=4,aspect=3).set(title='TRAIN Feature: Key')
sns.displot(data=test,x='key',color='darkred',height=4,aspect=3).set(title='TEST Feature: Key')
ax=sns.displot(data=train,x='key',height=4,aspect=3,hue='song_popularity',multiple="stack").set(title='Target Distribution')

In [89]:
# liveness plot
ax=sns.displot(data=train,x='liveness',kind='hist',color='Magenta',bins=50,height=4,aspect=3).set(title='TRAIN Feature: Liveness')
ax=sns.displot(data=test,x='liveness',kind='hist',color='purple',bins=50,height=4,aspect=3).set(title='TEST Feature: Liveness')
ax=sns.displot(data=train,x='liveness',kind='kde',height=4,aspect=4,hue='song_popularity').set(title='Target Distribution') 

In [90]:
# loudness plot
ax=sns.displot(data=train,x='loudness',kind='hist',color='Crimson',height=4,bins=50,aspect=3).set(title='TRAIN Feature: Loudness')
ax=sns.displot(data=test,x='loudness',kind='hist',color='green',height=4,bins=50,aspect=3).set(title='TEST Feature: Loudness')
ax=sns.displot(data=train,x='loudness',kind='kde',height=4,aspect=4,hue='song_popularity').set(title='Target Distribution') 

In [91]:
#audio mode
sns.displot(data=train,x='audio_mode',color='cyan',height=4,aspect=3).set(title='TRAIN Feature: Audio Mode') 
sns.displot(data=test,x='audio_mode',color='red',height=4,aspect=3).set(title='TEST Feature: Audio Mode') 
ax=sns.displot(data=train,x='audio_mode',height=4,aspect=3,hue='song_popularity',multiple="stack").set(title='Target Distribution') 

In [92]:
#Speechiness 
ax=sns.displot(data=train,x='speechiness',kind='hist',color='lightGreen',height=4,aspect=3).set(title='TRAIN Feature: Speechiness')
ax=sns.displot(data=test,x='speechiness',kind='hist',color='orange',height=4,aspect=3).set(title='TEST Feature: Speechiness')
ax=sns.displot(data=train,x='speechiness',kind='kde',height=4,aspect=4,hue='song_popularity').set(title='Target Distribution')

In [93]:
#tempo
ax=sns.displot(data=train,x='tempo',kind='hist',color='Black',bins=50,height=4,aspect=3).set(title='TRAIN Feature: Tempo')
ax=sns.displot(data=test,x='tempo',kind='hist',color='cyan',bins=50,height=4,aspect=3).set(title='TEST Feature: Tempo') 
ax=sns.displot(data=train,x='tempo',kind='kde',height=4,aspect=4,hue='song_popularity').set(title='Target Distribution') 

In [94]:
#Time signature
sns.displot(data=train,x='time_signature',color='DarkOrange',height=4,aspect=3).set(title='TRAIN Feature: Time Signature')
sns.displot(data=test,x='time_signature',color='Magenta',height=4,aspect=3).set(title='TEST Feature: Time Signature') 
ax=sns.displot(data=train,x='time_signature',height=4,aspect=3,hue='song_popularity',multiple="stack").set(title='Target Distribution')

In [95]:
#Audio Valence
ax=sns.displot(data=train,x='audio_valence',kind='hist',color='tan',height=4,aspect=3).set(title='TRAIN Feature: Instrumentalness') 
ax=sns.displot(data=test,x='audio_valence',kind='hist',color='black',height=4,aspect=3).set(title='TEST Feature: Instrumentalness') 
ax=sns.displot(data=train,x='audio_valence',kind='kde',height=4,aspect=4,hue='song_popularity').set(title='Target Distribution') 

In [96]:
#Song popularity
sns.displot(data=train,x='song_popularity',color='maroon',height=4,aspect=3).set(title='TRAIN Target: Song Popularity') 

In [11]:
fig, ax = plt.subplots(figsize=(20,10))
ax=sns.heatmap(train.corr(),annot=True,linewidths=.5)

In [9]:
#Max columns having nulls for any row
train_null=pd.DataFrame(train[train.isnull().any(axis=1)])
train_null.isna().sum(axis=1).max()

In [10]:
type(train_null)

In [11]:
train_null[train_null.isna().sum(axis=1)>=4]

In [12]:
#Dropping all the rows having null in 4 or more columns
null_idx=train[train.isna().sum(axis=1)>=4]
train.drop(null_idx.index,inplace=True)

In [13]:
train

In [14]:
#Tranforming columns intrumentalness in train and test
import numpy as np
train['log_instrumentalness']=train['instrumentalness'].apply(lambda x: np.log(x))
test['log_instrumentalness']=test['instrumentalness'].apply(lambda x: np.log(x))

In [15]:
#cols_to_be_transformed
cols_to_be_transformed=[]
for cols in train:
    if train[cols].isna().any():
        cols_to_be_transformed.append(cols)
cols_to_be_transformed

In [16]:
#number of nulls in each column
train[cols_to_be_transformed].isnull().sum().plot.barh(figsize=(6,4),color='red',position=1,width=0.2)
test[cols_to_be_transformed].isnull().sum().plot.barh(figsize=(8,8),color='blue',position=0,width=0.2)
plt.ylabel('Features containg NULLs')
plt.xlabel('Number of Nulls')
plt.title('Null Distribution over TRAIN and TEST')
plt.show()

In [17]:
inst_df=train[(train['instrumentalness'].isnull()==False)&(train['log_instrumentalness'].isnull()==True)]
inst_df

* So we see that by only taking log of the feature ***instrumentalness*** we increase the number of nulls even more, so next approach to try out is to split the feature into 2.**

In [18]:
train['part_b_instrumentalness']=np.where((train['instrumentalness'].isnull()==False) & (train['log_instrumentalness'].isnull()==True),0.0,train['log_instrumentalness'])
test['part_b_instrumentalness']=np.where((test['instrumentalness'].isnull()==False) & (test['log_instrumentalness'].isnull()==True),0.0,test['log_instrumentalness'])

In [19]:
#cols_to_be_transformed
cols_to_be_transformed=[]
for cols in train:
    if train[cols].isna().any():
        cols_to_be_transformed.append(cols)
cols_to_be_transformed

In [20]:
train[cols_to_be_transformed].isnull().sum().plot.barh(figsize=(6,4),color='red',position=1,width=0.2)
test[cols_to_be_transformed].isnull().sum().plot.barh(figsize=(8,8),color='blue',position=0,width=0.2)
plt.ylabel('Features containg NULLs')
plt.xlabel('Number of Nulls')
plt.title('Null Distribution over TRAIN and TEST')
plt.show()

In [21]:
#keeping only those intrumentalness value in intrumentalness where its log becomes null
train['part_a_instrumentalness']=np.where((train['part_b_instrumentalness']==0.0),train['instrumentalness'],0.0)
test['part_a_instrumentalness']=np.where((test['part_b_instrumentalness']==0.0),test['instrumentalness'],0.0)

In [22]:
train.info()

In [23]:
#Trying to add isna columns for all the coulumns which have null
train_bool=train[cols_to_be_transformed].isna().add_prefix('bool_')
train_bool['id']=train['id']
train_mod=train.merge(train_bool,how='left',left_on='id',right_on='id')
train_mod.info()

In [24]:
#Trying to add isna columns for all the coulumns which have null in test set
test_bool=test[cols_to_be_transformed].isna().add_prefix('bool_')
test_bool['id']=test['id']
test_mod=test.merge(test_bool,how='left',left_on='id',right_on='id')
test_mod.info()

In [25]:
#Dropping extra columns - instrumentalness, log_instrumentalness, bool_instrumentalness, bool_log_instrumentalness, id, song_popularity
y_train=train_mod['song_popularity']
train_mod.drop(['id','song_popularity','instrumentalness','log_instrumentalness','bool_instrumentalness','bool_log_instrumentalness'],axis=1,inplace=True)

In [26]:
#Dropping extra columns - instrumentalness, log_instrumentalness, bool_instrumentalness, bool_log_instrumentalness, id from test set
test_mod.drop(['id','instrumentalness','log_instrumentalness','bool_instrumentalness','bool_log_instrumentalness'],axis=1,inplace=True)

In [27]:
#cols_to_be_transformed
cols_to_be_transformed=[]
for cols in train_mod:
    if train_mod[cols].isna().any():
        cols_to_be_transformed.append(cols)
cols_to_be_transformed

In [28]:
train_mod[cols_to_be_transformed].isnull().sum().plot.barh(figsize=(6,4),color='red',position=1,width=0.2)
test_mod[cols_to_be_transformed].isnull().sum().plot.barh(figsize=(8,8),color='blue',position=0,width=0.2)
plt.ylabel('Features containg NULLs')
plt.xlabel('Number of Nulls')
plt.title('Null Distribution over TRAIN and TEST')
plt.show()

#Using 2 separate columns for instrumentalness, we reached a point where we got a smooth distribution and controlled the number of missing values

# # Imputation and Modeling

In [30]:
#Imputing data using KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

pipeline_knn=Pipeline([
    ('scale',StandardScaler()),
    ('impute',KNNImputer(n_neighbors=5)),
])

In [31]:
#Imputing data using IterativeImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

pipeline_iter=Pipeline([
    ('iter',IterativeImputer(random_state=42,imputation_order='descending'))
])

In [32]:
#Creating Stratified folds for cross-validation 
from sklearn.model_selection import StratifiedShuffleSplit

ss=StratifiedShuffleSplit(n_splits=5,test_size=0.2,random_state=42)

In [33]:
ss

In [34]:
#y_train=train_mod['song_popularity']
X_train=pd.DataFrame(train_mod)

In [35]:
X_train

# Trying RandomForest, SVC and KNN models with KNN Imputer

In [36]:
#Creating model
from sklearn.ensemble import RandomForestClassifier

impute_clf=RandomForestClassifier(n_estimators=50,random_state=42,max_depth=25)

In [37]:
#Model SVC
from sklearn.svm import SVC

svc_clf=SVC(random_state=42,C=1,tol=0.001,kernel='poly')

In [38]:
#Model KNN
from sklearn.neighbors import KNeighborsClassifier

knn_clf=KNeighborsClassifier(n_neighbors=10)

In [29]:
#Fitting the pipeline on one split, modeling and cross validation accuracy
from sklearn.metrics import roc_auc_score

def model_validate(model,pipeline,X_train,y_train):
    scores=[]
    for train_idx,val_idx in ss.split(X_train,y_train):
        X_train_new,X_val=X_train.iloc[train_idx],X_train.iloc[val_idx]
        y_train_new,y_val=y_train.iloc[train_idx],y_train.iloc[val_idx]
        print(pipeline)
        train_trans=pipeline.fit_transform(X_train_new)
        val_trans=pipeline.transform(X_val)
        model.fit(train_trans,y_train_new)
        val_pred=model.predict(val_trans)
        scores.append(roc_auc_score(y_val,val_pred))
    return scores

In [ ]:
#Forest_score with KNN Imputer -
forest_acc=model_validate(impute_clf,pipeline_knn,X_train,y_train)
forest_acc

In [171]:
#Forest_score with InterativeImputer -
forest_acc_iter=model_validate(impute_clf,pipeline_iter,X_train,y_train)
forest_acc_iter

In [ ]:
#SVC score with KNN Imuter
svc_acc=model_validate(svc_clf,piepline_knn,X_train,y_train)
svc_acc

In [169]:
#SVC score with IterativeImuter
svc_acc=model_validate(svc_clf,pipeline_iter,X_train,y_train)
svc_acc

In [ ]:
knn_clf.get_params()

In [ ]:
knn_acc=model_validate(knn_clf,X_train,y_train)
knn_acc

In [145]:
import xgboost as xgb

xgb_clf=xgb.XGBClassifier(random_state=42,n_estimators=50,max_depth=9,eval_metric='auc',alpha=0.06)
xgb_acc=model_validate(xgb_clf,pipeline_iter,X_train,y_train)
np.mean(xgb_acc)

In [94]:
X_train_iter=pipeline_iter.fit_transform(X_train)

In [160]:
xgb_clf=xgb.XGBClassifier(random_state=42,n_estimators=50,max_depth=9,eval_metric='auc',alpha=0.061)
xgb_clf.fit(X_train,y_train)

In [62]:
#Impute missing values and predict
X_test=pipeline_iter.transform(test_mod)

In [57]:
sub_xboost=pd.DataFrame(test['id'])

In [161]:
sub_xboost['song_popularity']=xgb_clf.predict(X_test)

In [162]:
sub_xboost['song_popularity'].value_counts()

In [163]:
sub_xboost.to_csv('submissionXGB2.csv')

In [ ]:
train[cols_to_be_transformed]

In [ ]:
train_trans=pipeline.fit_transform(train)

In [ ]:
len(train_trans)

In [ ]:
train_trans=pd.DataFrame(train_trans,columns=X_train.columns)

In [ ]:
train_trans.info()

In [ ]:
train_trans

In [ ]:
pipeline.get_params().keys()

In [ ]:
y_train=train['song_popularity']
X_train=pd.DataFrame(train)
X_train.drop(['id','instrumentalness','song_popularity'],axis=1,inplace=True)

In [ ]:
#Feeding this model into GridSearchCV
from sklearn.model_selection import GridSearchCV
params={'impute__n_neighbors':[5,10],'model__n_estimators':[50,100]}

grid_pipe=GridSearchCV(pipeline,params,cv=5,verbose=10,scoring='roc_auc')
grid_pipe.fit(X_train,y_train)

In [ ]:
#Setting up the pipeline with best features
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

pipeline=Pipeline([
    ('scale',StandardScaler()),
    ('impute',KNNImputer(n_neighbors=5)),
    ('model',RandomForestClassifier(n_estimators=100,random_state=42))
])
pipeline.fit_transform(X_train,y_train)

Hence from the above data we see that the songs in the range of 3mins are most popular

In [ ]:
#EDA after fitting the data -
ax=sns.displot(data=X_train_new,x='song_duration_ms',kind='hist',color='red',height=4,aspect=3)
ax=sns.displot(data=train_trans,x='song_duration_ms',kind='hist',color='red',height=4,aspect=3)

In [ ]:
# acousticness plot
import numpy as np
ax=sns.displot(data=X_train_new,x='acousticness',kind='hist',color='orange',height=5,aspect=3)
ax=sns.displot(data=train_trans,x='acousticness',kind='hist',color='orange',height=5,aspect=3)

In [ ]:
# danceability plot
ax=sns.displot(data=X_train_new,x='danceability',kind='hist',color='green',height=4,aspect=3)
ax=sns.displot(data=train_trans,x='danceability',kind='hist',color='green',height=4,aspect=3)
ax=sns.displot(data=train,x='danceability',kind='kde',height=4,aspect=3,hue='song_popularity')

In [ ]:
# energy plot
ax=sns.displot(data=X_train_new,x='energy',kind='hist',color='purple',height=4,aspect=3)           #kind=kde for distribution plot
ax=sns.displot(data=train_trans,x='energy',kind='hist',color='purple',height=4,aspect=3)
ax=sns.displot(data=train,x='energy',kind='kde',height=4,aspect=3,hue='song_popularity') 

In [ ]:
# instrumentalness plot
ax=sns.displot(data=train,x='instrumentalness',kind='kde',color='brown',height=4,aspect=3) 
ax=sns.displot(data=X_train_new,x='log_instrumentalness',kind='hist',color='brown',height=4,aspect=3) 
ax=sns.displot(data=train_trans,x='log_instrumentalness',kind='hist',color='brown',height=4,aspect=3)
ax=sns.displot(data=train,x='instrumentalness',kind='kde',height=4,aspect=3,hue='song_popularity') 

Using the log of the feature looks a reasonable outcome

In [ ]:
# key plot
sns.displot(data=X_train_new,x='key',color='darkblue',height=4,aspect=3)
sns.displot(data=train_trans,x='key',color='darkblue',height=4,aspect=3)
ax=sns.displot(data=train,x='key',height=4,aspect=3,hue='song_popularity',multiple="stack") 

In [ ]:
# liveness plot
ax=sns.displot(data=X_train_new,x='liveness',kind='hist',color='Magenta',height=4,aspect=3)
ax=sns.displot(data=train_trans,x='liveness',kind='hist',color='Magenta',height=4,aspect=3)
#ax=sns.displot(data=train,x=np.log(train['liveness']),kind='kde',color='Magenta',height=4,aspect=3)
ax=sns.displot(data=train,x='liveness',kind='kde',height=4,aspect=4,hue='song_popularity') 

Using the log of this feature as we get a good distribution using the latter

In [ ]:
# loudness plot
ax=sns.displot(data=X_train_new,x='loudness',kind='hist',color='Crimson',height=4,aspect=3)
ax=sns.displot(data=train_trans,x='loudness',kind='hist',color='Crimson',height=4,aspect=3)
ax=sns.displot(data=train,x='loudness',kind='kde',height=4,aspect=4,hue='song_popularity') 

There is a class imbalance, though not hugely imbalanced. But there is some imbalance between the classes

In [ ]:
train.info()

In [ ]:
#converting the songs in mins
train['song_duration_ms']=train['song_duration_ms']/(1000*60)

In [ ]:
train.rename(columns={'song_duration_ms':'song_duration_mins'},inplace=True)

In [ ]:
#Tranforming columns intrumentalness, liveness, speechiness
train['log_instrumentalness']=train['instrumentalness'].apply(lambda x: np.log(x))
#train['log_liveness']=train['liveness'].apply(lambda x: np.log(x))
#train['log_speechiness']=train['speechiness'].apply(lambda x: np.log(x))

In [ ]:
y_train=train['song_popularity']
X_train=pd.DataFrame(train)

In [ ]:
X_train

In [ ]:
X_train.drop(['id','instrumentalness','song_popularity'],axis=1,inplace=True)

In [ ]:
X_train

In [ ]:
import xgboost as xgb

xgb_clf=xgb.XGBClassifier(random_state=42,n_estimators=50,max_depth=10,learning_rate=0.38)
xgb_clf.fit(X_train[features_xgb],y_train)

In [ ]:
sub=test[['id']]
sub['song_popularity']=knn.predict(X_test)
sub['song_popularity'].value_counts()

In [ ]:
sub.to_csv('submission.csv')

In [ ]:
train.head()

In [ ]:
#Checking correlation of the columns - with the null values
fig, ax = plt.subplots(figsize=(20,10))
ax=sns.heatmap(X_train.corr(),annot=True,linewidths=.5)

In [ ]:
features_xgb=['acousticness','energy','loudness','log_instrumentalness']  

In [ ]:
train.describe()

In [ ]:
train['key'].value_counts()

In [ ]:
train.info()

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
ax=sns.heatmap(train.corr(),annot=True,linewidths=.5)

In [ ]:
#Sampling data
#count_of_non_popular_song=len(train[train['song_popularity']==0])
#popular_songs=train[train['song_popularity']==1]
#train_oversampled=popular_songs.sample(count_of_non_popular_song,replace=True)
#X_train_sampled=pd.concat([train_oversampled,train[train['song_popularity']==0]])

In [ ]:
train

In [ ]:
#Scaling the features 
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
pipeline=Pipeline([('std_scaler',StandardScaler())])

In [ ]:
#Using columns 'acousticness','energy','loudness','audio_valence','danceability','log_instrumentalness' as per the insight received from the above heatmap
#features_set1=['acousticness','energy','loudness','audio_valence','log_intrumentalness']
y_train=train['song_popularity']
X_train=pd.DataFrame(train)
X_train.drop(['instrumentalness','song_popularity','id'],axis=1,inplace=True)

In [ ]:
pipeline.fit_transform(X_train)

In [ ]:
train.info()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn=KNeighborsClassifier()

params={'weights':['distance'],'n_neighbors':[10,12,15]}

knn_grid=GridSearchCV(knn,params,verbose=10,cv=5,scoring='roc_auc')
knn_grid.fit(X_train,y_train)

In [ ]:
X_train

In [ ]:
sns.scatterplot(data=train, x="energy", y="loudness", hue="song_popularity")

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42)
over = SMOTE()
under = RandomUnderSampler()
steps = [('o', over), ('u', under),('model', model)]
pipeline_smote = Pipeline(steps=steps)

In [ ]:
from sklearn.model_selection import GridSearchCV
params={'model__n_estimators':[50,100,150],'model__max_features':[4,6]}

smote_cv=GridSearchCV(estimator=pipeline_smote,param_grid=params,cv=5,scoring='roc_auc',verbose=10)
smote_cv.fit(X_train,y_train)

In [ ]:
best_smote=smote_cv.best_estimator_.fit(X_train,y_train)

In [ ]:
smote_cv.get_params().keys()

In [ ]:
model.get_params()

In [ ]:
X_train,y_train=pipeline_smote.fit_resample(X_train,y_train)

In [ ]:
(y_train==0).count()

In [ ]:
svc_clf.get_params()

In [ ]:
#Creating a validation set
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC

svc_clf=LinearSVC(random_state=42)
param_svc={'C':[5,10],'tol':[0.005,0.05]}

svc_grid=GridSearchCV(svc_clf,param_svc,verbose=10,cv=5,scoring='roc_auc')
svc_grid.fit(X_train,y_train)

In [ ]:
svc_grid.best_estimator_

In [ ]:
#Building a model with best estimator
svc_final=svc_grid.best_estimator_
svc_final.fit(X_train,y_train)

In [ ]:
test=pd.read_csv('../input/song-popularity-prediction/test.csv')

In [ ]:
#Tweaking the time data 
test['song_duration_ms']=test['song_duration_ms']/1000*60
test.rename(columns={'song_duration_ms':'song_duration_mins'},inplace=True)

In [ ]:
#Creating log of feature instrumentalness
test['log_instrumentalness']=test['instrumentalness'].apply(lambda x: np.log(x))
#test['log_liveness']=test['liveness'].apply(lambda x: np.log(x))
#test['log_speechiness']=test['speechiness'].apply(lambda x: np.log(x))

In [ ]:
X_test=pd.DataFrame(test)
X_test

In [ ]:
X_test.drop(['id','instrumentalness'],inplace=True,axis=1)

In [ ]:
X_test

In [ ]:
#imputing the test data
test[cols_to_be_imputed_with_mean]=imputer.transform(test[cols_to_be_imputed_with_mean])
test['key']=impute_mode.transform(test['key'].to_numpy().reshape(-1,1))
test[cols_to_be_imputed_with_median]=impute_med.transform(test[cols_to_be_imputed_with_median])

In [ ]:
X_test=pipeline.transform(X_test)

In [ ]:
sub['song_popularity']=svc_final.predict(test)

In [ ]:
sub['song_popularity'].value_counts()

In [ ]:
ens1.get_params()

In [ ]:
#Running RandomForestRegressor on the whole dataset
X_train=train
X_train=pipeline.fit_transform(X_train)

In [ ]:
X_train

In [ ]:
from sklearn.ensemble import RandomForestClassifier

ens1=RandomForestClassifier(random_state=42)
param_grid={'n_estimators':[50,100,200],'max_features': [4,6,8]}

#grid_ens=GridSearchCV(ens1,param_grid,verbose=10,scoring='roc_auc',cv=10)
#grid_ens.fit(X_train,y_train)

In [ ]:
ens1.get_params()

In [ ]:
grid_ens.best_estimator_

In [ ]:
ens_final=RandomForestClassifier(max_features=6, random_state=42,n_estimators=50)
ens_final.fit(X_train,y_train)

In [ ]:
sub['song_popularity']=ens_final.predict(X_test)

In [ ]:
sub['song_popularity'].value_counts()

In [ ]:
y_val_pred=ens1.predict(X_val)

In [ ]:
#accuracy score
from sklearn.metrics import accuracy_score
accuracy_score(y_val,y_val_pred)

In [ ]:
test.info()

In [ ]:
test.drop(['instrumentalness','liveness','speechiness'],axis=1,inplace=True)

In [ ]:
test

In [ ]:
X_test=pipeline.transform(test)

In [ ]:
X_test

In [ ]:
sub=test[['id']]

In [ ]:
sub['song_popularity']=ens_final.predict(X_test)

In [ ]:
sub['song_popularity'].value_counts()   #50

In [ ]:
sub.to_csv('submission.csv')

In [ ]:
y_train=X_train_sampled['song_popularity']
#X_train_sampled.drop(['song_popularity'],axis=1,inplace=True)

In [ ]:
#X_train_sampled.drop(['instrumentalness','liveness','speechiness'],axis=1,inplace=True)

In [ ]:
#X_train_sampled.head()

In [ ]:
#from sklearn.ensemble import RandomForestClassifier

#ens1=RandomForestClassifier(random_state=42)
#param_grid={'n_estimators':[50,100,200],'max_features': [4,6,8]}

#grid_ens=GridSearchCV(ens1,param_grid,verbose=10,scoring='roc_auc',cv=5)
#grid_ens.fit(X_train_sampled,y_train)

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp_clf = MLPClassifier(random_state=42,max_iter=300)
mlp_clf.fit(X_train,y_train)

In [ ]:
sub['song_popularity']=final_tree.predict(X_test)

In [ ]:
sub['song_popularity'].value_counts()

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

ens1=ExtraTreesClassifier(random_state=42,class_weight='balanced')
param_grid={'n_estimators':[50,100,200],'max_features': [4,6,8]}

grid_ens=GridSearchCV(ens1,param_grid,verbose=10,scoring='roc_auc',cv=5)
grid_ens.fit(X_train,y_train)

In [ ]:
grid_ens.best_estimator_

In [ ]:
final_tree=ExtraTreesClassifier(max_features=6, n_estimators=50,random_state=42)
final_tree.fit(X_train,y_train)

In [ ]:
#Stacking Ensemble
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
random_forest_clf = RandomForestClassifier(n_estimators=50, random_state=42)
extra_trees_clf = ExtraTreesClassifier(n_estimators=50, random_state=42)
mlp_clf = MLPClassifier(random_state=42,max_iter=300)

In [ ]:
#Splitting the training data in train and val
from sklearn.model_selection import train_test_split

X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.2)

In [ ]:
estimators = [random_forest_clf, extra_trees_clf, mlp_clf]
for estimator in estimators:
    print("Training the", estimator)
    estimator.fit(X_train[features_xgb], y_train)

In [ ]:
#Predict the val output using each estimator and storing the prediction to fit a stack on the validation set
X_val_predictions = np.empty((len(X_val), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_val_predictions[:, index] = estimator.predict(X_val)

In [ ]:
rnf_blender=RandomForestClassifier(n_estimators=200,oob_score=True, random_state=42)
rnf_blender.fit(X_val_predictions,y_val)

In [ ]:
X_test_predictions = np.empty((len(test), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(test)

In [ ]:
sub['song_popularity']=rnf_blender.predict(X_test_predictions)

In [ ]:
sub['song_popularity'].value_counts()